In [1]:
import os
import math
import wandb
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *

from tensorflow.keras.regularizers import *

from tensorflow.keras.callbacks import *
from keras.preprocessing.image import *
from tensorflow.keras.preprocessing import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-21 23:35:47.832764: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 23:35:47.936672: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-21 23:35:47.961899: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory:

In [2]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [3]:
inputs = Input(shape=(32,32, 3))

# Layer 1
x = Conv2D(32, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(inputs)

# Layer 2
x = Conv2D(32, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 3
x = Conv2D(64, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(x)

# Layer 4
x = Conv2D(64, (3,3), activation='swish', kernel_regularizer=l2(1e-6))(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.1)(x)

# Layer 5
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512, activation='swish', kernel_regularizer=l2(1e-6))(x)

# Layer 6
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                             

2023-04-21 23:35:50.314653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 23:35:51.051285: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22277 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [4]:
# 모델 컴파일
opt = RMSprop(learning_rate=1e-4)

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
datagen.fit(x_train)

In [6]:
from tensorflow.keras import callbacks

wandb.init(project="CIFAR10", entity="hcim", name='LeNet-swish')

wandbCallback=wandb.keras.WandbCallback(monitor="val_accuracy", mode="max", 
                                        log_weights=True, log_gradients=True, 
                                        training_data=datagen.flow(x_train, y_train, batch_size=128))
        
    
callbacks = [wandbCallback]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chan4im (hcim). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [7]:
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=200, workers=4,
          callbacks=callbacks)

Epoch 1/200


2023-04-21 23:35:56.436450: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-21 23:35:57.707958: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


388/391 [============================>.] - ETA: 0s - loss: 2.1628 - accuracy: 0.1855

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best)... Done. 0.0s


...

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best)... Done. 0.0s


391/391 [==============================] - 22s 57ms/step - loss: 0.6419 - accuracy: 0.7753 - val_loss: 0.8344 - val_accuracy: 0.7174
Epoch 184/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6406 - accuracy: 0.7764 - val_loss: 0.8327 - val_accuracy: 0.7128
Epoch 185/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6374 - accuracy: 0.7749 - val_loss: 0.8330 - val_accuracy: 0.7152
Epoch 186/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6334 - accuracy: 0.7772 - val_loss: 0.8301 - val_accuracy: 0.7139
Epoch 187/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6292 - accuracy: 0.7774 - val_loss: 0.8250 - val_accuracy: 0.7150
Epoch 188/200
390/391 [============================>.] - ETA: 0s - loss: 0.6385 - accuracy: 0.7760

INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best/assets


INFO:tensorflow:Assets written to: /root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best/assets
wandb: Adding directory to artifact (/root/jupyter/IHC/gain_proj/wandb/run-20230421_233553-vb6akp5o/files/model-best)... Done. 0.0s


391/391 [==============================] - 22s 57ms/step - loss: 0.6385 - accuracy: 0.7761 - val_loss: 0.8213 - val_accuracy: 0.7218
Epoch 189/200
391/391 [==============================] - 20s 51ms/step - loss: 0.6340 - accuracy: 0.7782 - val_loss: 0.8327 - val_accuracy: 0.7147
Epoch 190/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6284 - accuracy: 0.7806 - val_loss: 0.8262 - val_accuracy: 0.7174
Epoch 191/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6325 - accuracy: 0.7800 - val_loss: 0.8278 - val_accuracy: 0.7185
Epoch 192/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6269 - accuracy: 0.7800 - val_loss: 0.8251 - val_accuracy: 0.7171
Epoch 193/200
391/391 [==============================] - 20s 51ms/step - loss: 0.6219 - accuracy: 0.7820 - val_loss: 0.8267 - val_accuracy: 0.7158
Epoch 194/200
391/391 [==============================] - 20s 52ms/step - loss: 0.6240 - accuracy: 0.7797 - val_loss: 0.8211 - val_ac